In [2]:
print("Hello")

Hello


In [1]:
from dotenv import load_dotenv
load_dotenv()  # Loads environment variables from .env file

from langchain_groq import ChatGroq  # ✅ correct import name

# Initialize your LLM (make sure GROQ_API_KEY is in your .env)
llm = ChatGroq(model="llama-3.1-8b-instant")  # You can change model name if needed

# Generate a simple response
response = llm.invoke("Hello, how are you?")

# Print the output
print(response.content)


I'm functioning properly, thanks for asking. How can I assist you today?


In [4]:
import sqlite3
import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool 

def get_engine_for_db():
    """The forst step is to pull the chinqook file and populate a local sqlite3 database.
    later on we will create a sqlalchemy engine to connect to the database.
    """
    
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text
    
    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    
    return create_engine("sqlite://", poolclass=StaticPool, creator=lambda: connection, connect_args={"check_same_thread": False})
    
engine = get_engine_for_db()
db = SQLDatabase(engine)

    

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore

checkpointer = MemorySaver()
store = InMemoryStore()

In [8]:
from typing_extensions import TypedDict
from typing import List, Annotated
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.managed.is_last_step import RemainingSteps

class State(TypedDict):
    """State representation for the multi-agent system."""
    customer_id: str
    messages: Annotated[List[AnyMessage], add_messages]
    loaded_memory:str
    remaining_steps: RemainingSteps

In [12]:
from langchain_core.tools import tool
import ast

@tool
def get_album_by_artist_name(artist:str) ->str:
    """Get album names by artist name from the database."""
    return db.run(f"""
    SELECT Album.Title, Artist.Name 
        FROM Album 
        JOIN Artist ON Album.ArtistId = Artist.ArtistId 
        WHERE Artist.Name LIKE '%{artist}%';
    """, 
    include_column_names=True
    )
    
    
@tool
def get_song_by_artist_name(artist:str) ->str:
    """Get songs by an artist (or similar artists)."""
    return db.run(
        f"""
        SELECT Track.Name as SongName, Artist.Name as ArtistName 
        FROM Album 
        LEFT JOIN Artist ON Album.ArtistId = Artist.ArtistId 
        LEFT JOIN Track ON Track.AlbumId = Album.AlbumId 
        WHERE Artist.Name LIKE '%{artist}%';
        """,
        include_columns=True
    )
    
@tool
def check_for_songs(song_title):
    """Check if a song exists by its name."""
  
    return db.run(
        f"""
        SELECT * FROM Track WHERE Name LIKE '%{song_title}%';
        """,
        include_columns=True
    )
    

@tool
def get_song_by_genre(genre:str) ->str:
    """Get songs by genre from the database."""
    genre_id_query = f"SELECT GenreId FROM Genre WHERE Name LIKE '%{genre}%'"
    genre_ids = db.run(genre_id_query)
    
    if not genre_ids:
        return f"No songs found for the genre: {genre}"
    
    genre_ids = ast.literal_eval(genre_ids)
    genre_id_list = ", ".join(str(gid[0]) for gid in genre_ids)

    songs_query = f"""
        SELECT Track.Name as SongName, Artist.Name as ArtistName
        FROM Track
        LEFT JOIN Album ON Track.AlbumId = Album.AlbumId
        LEFT JOIN Artist ON Album.ArtistId = Artist.ArtistId
        WHERE Track.GenreId IN ({genre_id_list})
        GROUP BY Artist.Name
        LIMIT 8;
    """
    songs = db.run(songs_query, include_columns=True)
    
    if not songs:
        return f"No songs found for the genre: {genre}"
        

    formatted_songs = ast.literal_eval(songs)
    return [
        {"Song": song["SongName"], "Artist": song["ArtistName"]}
        for song in formatted_songs
    ]
    
music_tools = [
    get_album_by_artist_name,
    get_song_by_artist_name,
    check_for_songs,
    get_song_by_genre]

llm_with_tools = llm.bind_tools(music_tools)


In [14]:
from langgraph.prebuilt import ToolNode
music_ToolNode = ToolNode(music_tools)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.runnables import RunnableConfig

def generate_music_assistant_prompt(memory: str = "None") -> str:
    return f"""
    You are a member of the assistant team, your role specifically is to focused on helping customers discover and learn about music in our digital catalog. 
    If you are unable to find playlists, songs, or albums associated with an artist, it is okay. 
    Just inform the customer that the catalog does not have any playlists, songs, or albums associated with that artist.
    You also have context on any saved user preferences, helping you to tailor your response. 
    
    CORE RESPONSIBILITIES:
    - Search and provide accurate information about songs, albums, artists, and playlists
    - Offer relevant recommendations based on customer interests
    - Handle music-related queries with attention to detail
    - Help customers discover new music they might enjoy
    - You are routed only when there are questions related to music catalog; ignore other questions. 
    
    SEARCH GUIDELINES:
    1. Always perform thorough searches before concluding something is unavailable
    2. If exact matches aren't found, try:
       - Checking for alternative spellings
       - Looking for similar artist names
       - Searching by partial matches
       - Checking different versions/remixes
    3. When providing song lists:
       - Include the artist name with each song
       - Mention the album when relevant
       - Note if it's part of any playlists
       - Indicate if there are multiple versions
    
    Additional context is provided below: 

    Prior saved user preferences: {memory}
    
    Message history is also attached.  
    """
    
    
def music_assistant(state: State, config: RunnableConfig): 

 
    memory = "None" 
    if "loaded_memory" in state: 
        memory = state["loaded_memory"]

    music_assistant_prompt = generate_music_assistant_prompt(memory)

    response = llm_with_music_tools.invoke([SystemMessage(music_assistant_prompt)] + state["messages"])
    
    return {"messages": [response]}